# Apache bean solution for data exploring

### Creado por: Yohan Alfonso Hernandez
### Fecha: 26-05-2023

Lest explore a clean data using the file "Clean_df" used in the project 'https://github.com/yohan-alfonso/Exploracion_Datos' the diference is thta in this project we are not gointo to plot just make reading and transforming since the propouse is to practice a little bit of Beam proraming

- In this part we are goin to create the reading and transformations of the dataset
- We are goin to sent the out put to a file or you can print it, if you are going to pirnt the resulta you have to uncomment the line

**Create Functions**

In [5]:
import apache_beam as beam
import pandas as pd
import matplotlib.pyplot as plt
import apache_beam.dataframe as df
import apache_beam as beam

from apache_beam.io import ReadFromText, WriteToText
import apache_beam.runners.interactive.interactive_beam as ib


class split_and_lowercase(beam.DoFn):
    def process(self, df):
        # Split the element into individual words
        words = df.split(',')
        # Convert each word to lowercase
        lowercased_words = [word.lower() for word in words]
        # Return the lowercase words as a list
        return [lowercased_words]
    
class filter_mean_salary(beam.DoFn):
    def process(self, column):
        column_value = column[3]
        if column_value != '0.0':
            return [column]

class segregate_dataset(beam.DoFn):
    def process(self, df):
        select_columns = (df[1],df[2],df[3],df[4]+' '+df[7])
        yield select_columns
        
def try_float(value):
    try:
        return float(value)
    except ValueError:
        return None
    
def format_row(row):
    return [row['Column1'], row['Column2'], row['Column3'], row['Column4']]  

def run_pipeline(input_files):
    # Create the Pipeline object
    with beam.Pipeline() as pipeline:
        # Read the input files into a PCollection
        lines = pipeline | "ReadFiles" >> ReadFromText(input_files)
        
        
        transformed_data = (
        lines 
         # Split each line by commas
         # Split each line into words and convert to lowercase
        | "SplitAndLowercase" >> beam.ParDo(split_and_lowercase())
        
        # filter column mean salary diferent from the string '0.0'
        | "filter column" >> beam.ParDo(filter_mean_salary()) 
        
   
        # Segregate data that is considered useful for analisis and concatenate period and jobtype for simplicity
        | "segregate data" >>  beam.ParDo(segregate_dataset())
        
        | 'TransformData' >> beam.Map(lambda columns: {'Column1': try_float(columns[0]), 'Column2': try_float(columns[1]), 'Column3': try_float(columns[2]),'Column4': columns[3]})
        
        )
             
        transformed_data | beam.Map(print)
        
    #Write the split and filtered lines to the output file
    #salida_data = Segregate_data | "WriteToFile" >> WriteToText(output_file, file_name_suffix='.csv')

# Specify the input and output files
input_files = 'clean_df.csv'
#output_file = 'datos_segregados'

# Run the pipeline
run_pipeline(input_files)





{'Column1': None, 'Column2': None, 'Column3': None, 'Column4': 'period job_type'}
{'Column1': 900.0, 'Column2': 1300.0, 'Column3': 1100.0, 'Column4': 'hour full time'}
{'Column1': 8000000.0, 'Column2': 9500000.0, 'Column3': 8750000.0, 'Column4': 'year full time'}
{'Column1': 7000000.0, 'Column2': 10000000.0, 'Column3': 8500000.0, 'Column4': 'year full time'}
{'Column1': 6800000.0, 'Column2': 7200000.0, 'Column3': 7000000.0, 'Column4': 'year full time temporary'}
{'Column1': 5800000.0, 'Column2': 6500000.0, 'Column3': 6150000.0, 'Column4': 'year full time'}
{'Column1': 3200000.0, 'Column2': 0.0, 'Column3': 3200000.0, 'Column4': ' full time'}
{'Column1': 4500000.0, 'Column2': 10000000.0, 'Column3': 7250000.0, 'Column4': 'year full time'}
{'Column1': 4000000.0, 'Column2': 5000000.0, 'Column3': 4500000.0, 'Column4': 'year full time'}
{'Column1': 1375.0, 'Column2': 1675.0, 'Column3': 1525.0, 'Column4': 'hour full time'}
{'Column1': 3500000.0, 'Column2': 4500000.0, 'Column3': 4000000.0, 'Col